# Metehan Ayhan - Skin Cancer Classification Project

Bu projede kanserli ve kanser olmayan cilt resimleri kullanılarak kendisine verilen cilt resmiyle, cildin kanserli olup olmadığını tahmin eden bir yapay zeka modeli geliştireceğiz.

### Gerekli Paketleri import edelim

In [5]:
import cv2
import pandas as pd
import os #bu paket dosyaları okuma açma işlemi için..

In [6]:
#pip install opencv-python

In [7]:
labels=['Cancer','Non_Cancer']
img_path='Skin_Data/' #resimlerin yolunu verdik

In [8]:
os.listdir('Skin_Data')

['Cancer', 'Non_Cancer']

In [9]:
img_list=[] # önce boş liste oluşturduk resimleri okudukça içine atacağız.
label_list=[] #etiketler

for label in labels:
    for img_file in os.listdir(img_path+label):
        img_list.append(img_path+label+'/'+img_file)
        label_list.append(label)

In [10]:
df=pd.DataFrame({'img':img_list, 'label':label_list})

In [11]:
df.head()

,img,label
0,Skin_Data/Cancer/1007-1.jpg,Cancer
1,Skin_Data/Cancer/1010-01.JPG,Cancer
2,Skin_Data/Cancer/1012-2.JPG,Cancer
3,Skin_Data/Cancer/1031-1.jpg,Cancer
4,Skin_Data/Cancer/1051-3(94).jpg,Cancer


In [12]:
df.tail()

,img,label
283,Skin_Data/Non_Cancer/953-1.JPG,Non_Cancer
284,Skin_Data/Non_Cancer/954-3.JPG,Non_Cancer
285,Skin_Data/Non_Cancer/955.JPG,Non_Cancer
286,Skin_Data/Non_Cancer/984.JPG,Non_Cancer
287,Skin_Data/Non_Cancer/986-1.JPG,Non_Cancer


In [13]:
 import matplotlib.pyplot as plt

In [14]:
d={'Cancer':1,'Non_Cancer':0}

In [15]:
df['encode_label']=df['label'].map(d)

In [16]:
df.sample(5)

,img,label,encode_label
25,Skin_Data/Cancer/1777-1.JPG,Cancer,1
125,Skin_Data/Non_Cancer/1393-1.JPG,Non_Cancer,0
156,Skin_Data/Non_Cancer/1834-01.JPG,Non_Cancer,0
90,Skin_Data/Non_Cancer/1054.JPG,Non_Cancer,0
263,Skin_Data/Non_Cancer/783.JPG,Non_Cancer,0


In [17]:
import numpy as np

In [18]:
# normalize edelim
x=[]
for img in df['img']:
    img=cv2.imread(img)   #resimleri okuma
    img=cv2.resize(img,(170,170))   #yeniden boyutlandırma 179x170 pixel
    img=img/255.0   #normalize
    x.append(img)

In [19]:
 x=np.array(x) #sayı dizisine dönüştürüyoruz..

In [20]:
y=df['encode_label']

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=.20,random_state=42)

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Input, MaxPooling2D, Dropout, Reshape, BatchNormalization

### CNN - Convolutional Neural Networks

In [25]:
model=Sequential()
model.add(Input(shape=(170,170,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])          

In [26]:
history= model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=15,verbose=1)

Epoch 1/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 443ms/step - accuracy: 0.6827 - loss: 17.2804 - val_accuracy: 0.7414 - val_loss: 0.7153
Epoch 2/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 362ms/step - accuracy: 0.6621 - loss: 1.3574 - val_accuracy: 0.6034 - val_loss: 0.6417
Epoch 3/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 394ms/step - accuracy: 0.7476 - loss: 0.5569 - val_accuracy: 0.7414 - val_loss: 0.6001
Epoch 4/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 356ms/step - accuracy: 0.7229 - loss: 0.5588 - val_accuracy: 0.4655 - val_loss: 0.8085
Epoch 5/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 342ms/step - accuracy: 0.6863 - loss: 0.6313 - val_accuracy: 0.7759 - val_loss: 0.5664
Epoch 6/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 352ms/step - accuracy: 0.7808 - loss: 0.4980 - val_accuracy: 0.7069 - val_loss: 0.6426
Epoch 7/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 366ms/step - accuracy: 0.8030 - loss: 0.4818 - val_accuracy: 0.7931 - val_loss: 0.5140
Epoch 8/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 370ms/step - accuracy: 0.8333 - loss: 0.4147 - val_accuracy: 0.8103 - val_loss

In [27]:
model.save('my_cnn_model.h5')